In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets,transforms

In [2]:
import torch.nn.functional as F

In [3]:
from torch.utils.data import DataLoader

In [4]:
import matplotlib.pyplot as plt

In [5]:
from torchvision import models
import os

In [6]:
from PIL import Image
from IPython.display import display

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
train_transforms=transforms.Compose([transforms.RandomRotation(10),
                              transforms.RandomHorizontalFlip(),
                              transforms.Resize(224),
                              transforms.CenterCrop(224),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

In [9]:
test_transforms=transforms.Compose([transforms.Resize(224),
                              transforms.CenterCrop(224),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

In [12]:
cd Documents

C:\Users\rohan\Documents


In [33]:
train_loader=DataLoader(train_data,batch_size=10,shuffle=True)
test_loader=DataLoader(test_data,batch_size=10,shuffle=False)

In [14]:
train_data=datasets.ImageFolder('CATS_DOGS/train/',transform=train_transforms)

In [15]:
test_data=datasets.ImageFolder('CATS_DOGS/test/',transform=test_transforms)

In [16]:
class_names=train_data.classes

In [25]:
model=models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\rohan/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth


In [26]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [27]:
for parameters in model.parameters():
    parameters.requires_grad=False

In [28]:
model.classifier=nn.Sequential(nn.Linear(9216,1024),nn.ReLU(),nn.Dropout(0.5),nn.Linear(1024,2),nn.LogSoftmax(dim=1))

In [29]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, 

In [31]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.classifier.parameters(),lr=0.001)

In [34]:
epochs=2
train_losses=[]
predictedp=[]

for i in range(epochs):
    batch_corr=0
    for b,(X_train,y_train) in enumerate(train_loader):
        y_val=model(X_train)
        loss=criterion(y_val,y_train)
        predicted=torch.max(y_val.data,1)[1]
        batch_corr=(predicted==y_train).sum()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_losses.append(loss)
    predictedp.append(predicted)

In [35]:
train_losses

[tensor(0.0095, grad_fn=<NllLossBackward>),
 tensor(0.0080, grad_fn=<NllLossBackward>)]

In [36]:
test_loader_all=DataLoader(test_data,batch_size=10000,shuffle=False)
with torch.no_grad():
    for b,(X_test,y_test) in enumerate(test_loader_all):
        io=model(X_test)
        lossees=criterion(io,y_test)

In [37]:
lossees

tensor(0.1696)